In [1]:
import sparknlp
 
spark = sparknlp.start() # for GPU training >> sparknlp.start(gpu = True) # for Spark 2.3 =>> sparknlp.start(spark23 = True)
import pyspark.sql.functions as F
from sparknlp.annotator import *
from sparknlp.base import *
import sparknlp
from sparknlp.pretrained import PretrainedPipeline
 
print("Spark NLP version", sparknlp.version())
 
print("Apache Spark version:", spark.version)

Spark NLP version 3.3.4
Apache Spark version: 3.1.2


In [2]:
from pyspark.sql import SparkSession
import pyspark.sql.functions as fn
spark.sparkContext.appName

'Spark NLP'

In [3]:
from pyspark.sql.functions import *
from pyspark.sql.types import TimestampType

In [4]:
import pandas as pd
import numpy as np
from datetime import *

In [5]:
spark.conf.set("spark.sql.execution.arrow.pyspark.enabled", True)
#ps.set_option("compute.default_index_type", "distributed")

In [84]:
reviews = pd.read_csv('_data/Dataset_Review(F).csv')
reviews['date']=pd.to_datetime(reviews['date'])
reviews['date']=reviews['date'].
print(reviews.columns)



In [75]:
tmp = pd.read_csv('_data/Dataset_TMP_violation_level(F).csv')

tmp['date'] = pd.to_datetime(tmp['violdttm'])

print(tmp.columns)

len(tmp)


Index(['Unnamed: 0', 'businessname', 'Business_ID', 'violdttm', 'TMP_ID', '*',
       '**', '***', 'date'],
      dtype='object')


89155

In [81]:

# def flatten_reviews(label_df, reviews):
#     """ label_df: inspection dataframe with date, business_id
#         reviews: dataframe of reviews
        
#         Returns all of the text of reviews previous to each
#         inspection listed in label_df.
#     """
#     reviews_dictionary = {}
#     N = len(label_df)
    
#     # 將所有review按照時間區間框選出來, 製作成pre_inspection_reviews, 將所有item用空白join起來, 以all_text存入對應的pid
#     for i, (pid, row) in enumerate(label_df.iterrows()):
#         # we want to only get reviews for this restaurant that ocurred before the inspection
#         pre_inspection_mask = (reviews.date < row.date) & (reviews.business_id == row.Business_ID)
        
#         # pre-inspection reviews
#         pre_inspection_reviews = reviews[pre_inspection_mask]
        
#         # join the text
#         all_text = ' '.join(pre_inspection_reviews.text)

#         # store in dictionary
#         reviews_dictionary[pid] = all_text
        
        
#         if i % 2500 == 0:
#             print('{} out of {}'.format(i, N))
        
#     # return series in same order as the original data frame
#     return pd.Series(reviews_dictionary)[label_df.index]

# train_text = flatten_reviews(tmp, reviews)
# train_text.head()

0 out of 89155
2500 out of 89155
5000 out of 89155
7500 out of 89155
10000 out of 89155
12500 out of 89155
15000 out of 89155
17500 out of 89155
20000 out of 89155
22500 out of 89155
25000 out of 89155
27500 out of 89155
30000 out of 89155
32500 out of 89155
35000 out of 89155
37500 out of 89155
40000 out of 89155
42500 out of 89155
45000 out of 89155
47500 out of 89155
50000 out of 89155
52500 out of 89155
55000 out of 89155
57500 out of 89155
60000 out of 89155
62500 out of 89155
65000 out of 89155
67500 out of 89155
70000 out of 89155
72500 out of 89155
75000 out of 89155
77500 out of 89155
80000 out of 89155
82500 out of 89155
85000 out of 89155
87500 out of 89155


0    Great spot for lunch! The food was delicious a...
1    Great spot for lunch! The food was delicious a...
2    Great spot for lunch! The food was delicious a...
3    Great spot for lunch! The food was delicious a...
4    Great spot for lunch! The food was delicious a...
dtype: object

In [6]:
sdf = spark.read.csv("file:/home/john/_data/tmp_all_text_89155.csv",inferSchema=True,header=True)
sdf.show(2)


+---+----------+--------------------+--------------------+----------+------+---+---+---+----------+--------------------+
|_c0|Unnamed: 0|        businessname|         Business_ID|  violdttm|TMP_ID|  *| **|***|      date|            all_text|
+---+----------+--------------------+--------------------+----------+------+---+---+---+----------+--------------------+
|  0|         0|100% Delicias Exp...|dAolyz6VhWLFnpltq...|2019/05/07|   172|  2|  0|  2|2019-05-07|Great spot for lu...|
|  1|         1|100% Delicias Exp...|dAolyz6VhWLFnpltq...|2020/03/18|   181|  7|  1|  1|2020-03-18|Great spot for lu...|
+---+----------+--------------------+--------------------+----------+------+---+---+---+----------+--------------------+
only showing top 2 rows



In [15]:
x=0.25
subset_df, large_df = sdf.randomSplit([x, 1 - x])
subset_count = subset_df.count()
print(subset_count)

22098


In [16]:
import nltk
# Build NLP preprocessing pipeline
from sparknlp.base import DocumentAssembler
document_assembler = DocumentAssembler() \
.setInputCol('all_text') \
.setOutputCol('document')
from sparknlp.annotator import Tokenizer
tokenizer = Tokenizer() \
.setInputCols(['document']) \
.setOutputCol('tokenized') \
.setContextChars(['(', ')']) \
.setSplitChars(['-'])
from sparknlp.annotator import Normalizer
normalizer = Normalizer() \
.setInputCols(['tokenized']) \
.setOutputCol('normalized') \
.setLowercase(True) \
.setCleanupPatterns(['[^A-Za-z]'])
from sparknlp.annotator import LemmatizerModel
lemmatizer = LemmatizerModel \
.pretrained() \
.setInputCols(['normalized']) \
.setOutputCol('lemmatized')
from nltk.corpus import stopwords
nltk_stopwords = stopwords.words('english')
from sparknlp.annotator import StopWordsCleaner
stopwords_cleaner = StopWordsCleaner() \
.setInputCols(['lemmatized']) \
.setOutputCol('unigrams') \
.setStopWords(nltk_stopwords)
from sparknlp.annotator import NGramGenerator
ngrammer = NGramGenerator() \
    .setInputCols(['unigrams']) \
    .setOutputCol('ngrams') \
    .setN(2) \
    .setEnableCumulative(True) \
    .setDelimiter('_')
from sparknlp.base import Finisher
finisher = Finisher() \
.setInputCols(['unigrams', 'ngrams'])

lemma_antbnc download started this may take some time.
Approximate size to download 907.6 KB
[OK!]


In [17]:
# Assemble pipeline
from pyspark.ml import Pipeline
pipeline = Pipeline() \
.setStages([document_assembler,
            tokenizer,
            normalizer,
            lemmatizer,
            stopwords_cleaner,
            ngrammer,
            finisher])

In [88]:
stmp = spark.createDataFrame(tmp)

/home/john/.local/lib/python3.6/site-packages/pyspark/sql/pandas/conversion.py:289: UserWarning: createDataFrame attempted Arrow optimization because 'spark.sql.execution.arrow.pyspark.enabled' is set to true; however, failed by the reason below:
  PyArrow >= 1.0.0 must be installed; however, it was not found.
Attempting non-optimization as 'spark.sql.execution.arrow.pyspark.fallback.enabled' is set to true.
  warnings.warn(msg)


In [90]:

stmp[['all_text']].show(2)

+--------------------+
|            all_text|
+--------------------+
|Great spot for lu...|
|Great spot for lu...|
+--------------------+
only showing top 2 rows



In [134]:
#x=0.25
#subset_df, large_df = stmp.randomSplit([x, 1 - x])
#subset_count = subset_df.count()
#print(subset_count)

In [18]:
processed_reviews = pipeline.fit(subset_df).transform(subset_df)

In [19]:
processed_reviews.take(1)

[Row(_c0=9, Unnamed: 0=9, businessname='100% Delicias Express', Business_ID='dAolyz6VhWLFnpltqOWp1w', violdttm='2019/10/28', TMP_ID=211, *=0, **=0, ***=0, date='2019-10-28', all_text="Great spot for lunch! The food was delicious and fresh with super generous portions. The rice with chicken stew could feed two. The beef empanadas was amazing and juicy and I loved the papaya smoothie. The only thing that initially made me hesitate to even order was the clinical smell (chemicals from cleaning stuff?) and the interior isn't very inviting to eat-in  not quite cozy but perfect for take out! Their pork mofongo brought tears to our eyes. Literally...like  this was the best Mofongo we've ever had.Let's go back to the beginning. I'm perusing a local JP facebook page  and I see a post about a new Dominican restaurant opening up on the main drag in JP. My fiance is in town  and aside from always getting sushi  we like to eat Mofongo. Good mofongo. I mean  we'll eat anything really...but we can't p

In [20]:
print(processed_reviews.columns)
processed_reviews[['finished_unigrams','finished_ngrams']].show(2)

['_c0', 'Unnamed: 0', 'businessname', 'Business_ID', 'violdttm', 'TMP_ID', '*', '**', '***', 'date', 'all_text', 'finished_unigrams', 'finished_ngrams']
+--------------------+--------------------+
|   finished_unigrams|     finished_ngrams|
+--------------------+--------------------+
|[great, spot, lun...|[great, spot, lun...|
|[great, spot, lun...|[great, spot, lun...|
+--------------------+--------------------+
only showing top 2 rows



In [21]:
(trainingData, testData) = processed_reviews.randomSplit([0.8, 0.2])

In [24]:
trainingData_count = subset_count * 0.8
print(trainingData_count)

17678.4


In [25]:
from pyspark.ml.feature import CountVectorizer
from pyspark.ml.feature import IDF

In [66]:
tfizer = CountVectorizer(inputCol = 'finished_ngrams', outputCol = 'tf_features', 
                         minDF = 0.01, maxDF = 0.1, vocabSize = int(trainingData_count / 2))

tf_model = tfizer.fit(trainingData)
tf_result_training = tf_model.transform(trainingData)
tf_result_test = tf_model.transform(testData)

idfizer = IDF(inputCol = 'tf_features', outputCol = 'tfidf_features')

idf_model = idfizer.fit(tf_result_training)
tfidf_result_training = idf_model.transform(tf_result_training)
tfidf_result_test = idf_model.transform(tf_result_test)

In [33]:
tf_model.save("_data/tmp_tfModel.model")
idf_model.save("_data/tmp_idfModel.model")

In [34]:
print(tfidf_result_training.show(2))
print(tfidf_result_test.show(2))

+---+----------+--------------------+--------------------+----------+------+---+---+---+----------+--------------------+--------------------+--------------------+--------------------+--------------------+
|_c0|Unnamed: 0|        businessname|         Business_ID|  violdttm|TMP_ID|  *| **|***|      date|            all_text|   finished_unigrams|     finished_ngrams|         tf_features|      tfidf_features|
+---+----------+--------------------+--------------------+----------+------+---+---+---+----------+--------------------+--------------------+--------------------+--------------------+--------------------+
|  9|         9|100% Delicias Exp...|dAolyz6VhWLFnpltq...|2019/10/28|   211|  0|  0|  0|2019-10-28|Great spot for lu...|[great, spot, lun...|[great, spot, lun...|(8839,[39,184,243...|(8839,[39,184,243...|
| 10|        10|100% Delicias Exp...|dAolyz6VhWLFnpltq...|2020/06/10|   220|  7|  1|  1|2020-06-10|Great spot for lu...|[great, spot, lun...|[great, spot, lun...|(8839,[39,184,243.

In [35]:
from pyspark.ml.feature import CountVectorizerModel
tf_model = CountVectorizerModel.load("file:/home/john/_data/tmp_tfModel.model")

In [36]:
tf_model.vocabulary

['baseball',
 'stadium',
 'lobster_roll',
 'dim_sum',
 'museum',
 'fenway_park',
 'indian',
 'waffle',
 'field',
 'tapa',
 'korean',
 'exhibit',
 'ballpark',
 'yankee',
 'raman',
 'gnocchi',
 'pad_thai',
 'pho',
 'truffle',
 'japanese',
 'legal',
 'monster',
 'hot_dog',
 'crab_cake',
 'lobby',
 'maki',
 'restaurant_week',
 'latte',
 'sashimi',
 'french_toast',
 'cart',
 'falafel',
 'risotto',
 'sox_fan',
 'filet',
 'front_desk',
 'green_monster',
 'scallion',
 'italian_food',
 'el',
 'miso',
 'fig',
 'tempura',
 'watch_game',
 'fish_chip',
 'guacamole',
 'hummus',
 'valet',
 'sam_adam',
 'short_rib',
 'historic',
 'naan',
 'shabu',
 'shower',
 'benedict',
 'whole_food',
 'indian_food',
 'prosciutto',
 'jazz',
 'veal',
 'toro',
 'mexican_food',
 'coffee_shop',
 'bouncer',
 'baseball_fan',
 'hash',
 'grill_cheese',
 'live_music',
 'creme',
 'omelette',
 'spring_roll',
 'chipotle',
 'mfa',
 'newbury_street',
 'vietnamese',
 'pita',
 'hot_pot',
 'theater',
 'bark',
 'eastern',
 'squid',
 '

In [37]:
from pyspark.ml.clustering import LDA
num_topics = 10
max_iter = 10
lda = LDA(k = num_topics, 
          maxIter = max_iter, 
          featuresCol = 'tfidf_features')
ldaModel = lda.fit(tfidf_result_training)

In [38]:
from pyspark.sql import types as T
from pyspark.sql import functions as F
vocab = tf_model.vocabulary
def get_words(token_list):
    return [vocab[token_id] for token_id in token_list]
udf_to_words = F.udf(get_words, T.ArrayType(T.StringType()))

In [39]:
# 取10個字詞
num_top_words = 10
topics = ldaModel \
.describeTopics(num_top_words) \
.withColumn('topicWords', udf_to_words(F.col('termIndices')))
topics.select('topic', 'topicWords').show(truncate = 100)

+-----+----------------------------------------------------------------------------------------------------+
|topic|                                                                                          topicWords|
+-----+----------------------------------------------------------------------------------------------------+
|    0|          [museum, stadium, exhibit, baseball, fenway_park, ballpark, mfa, field, yankee, bark_crab]|
|    1|      [indian, giacomos, naan, indian_food, italian_food, lobby, masala, tikka, gnocchi, front_desk]|
|    2|                      [maki, sashimi, douzo, raman, japanese, miso, tempura, boba, udon, good_sushi]|
|    3|[dim_sum, soup_dumpling, taiwanese, bao, scallion, scallion_pancake, dumpling_house, taiwan, shab...|
|    4|      [fig, good_pizza, celtics, td, prosciutto, pizza_place, arena, bruins, pizza_good, thin_crust]|
|    5|[pho, bouncer, crepe, live_music, oyster_house, old_restaurant, whiskey, union, tavern, union_oys...|
|    6| [mexican_fo

In [ ]:
#from pyspark.ml.feature import VectorAssembler
#vectorAssembler = VectorAssembler(inputCols = ['CRIM', 'ZN', 'INDUS', 'CHAS', 'NOX', 'RM', 'AGE', 'DIS', 'RAD', 'TAX', 'PT', 'B', 'LSTAT'], outputCol = 'features')
#vhouse_df = vectorAssembler.transform(house_df)
#vhouse_df = vhouse_df.select(['features', 'MV'])
#vhouse_df.show(3)

In [67]:
tfidf_result_test = tfidf_result_test.withColumnRenamed('***','level_3')
tfidf_result_test = tfidf_result_test.withColumnRenamed('**','level_2')
tfidf_result_test = tfidf_result_test.withColumnRenamed('*','level_1')
tfidf_result_training = tfidf_result_training.withColumnRenamed('***','level_3')
tfidf_result_training = tfidf_result_training.withColumnRenamed('**','level_2')
tfidf_result_training = tfidf_result_training.withColumnRenamed('*','level_1')
#({'*':'level_1','**':'level_2','***':'level_3'})
#tfidf_result_training.show(3)

In [61]:
### *
tfidf_result_test.show(1)

+---+----------+--------------+--------------------+----------+------+---+---+---+----------+--------------------+--------------------+--------------------+--------------------+--------------------+
|_c0|Unnamed: 0|  businessname|         Business_ID|  violdttm|TMP_ID|  *| **|***|      date|            all_text|   finished_unigrams|     finished_ngrams|         tf_features|      tfidf_features|
+---+----------+--------------+--------------------+----------+------+---+---+---+----------+--------------------+--------------------+--------------------+--------------------+--------------------+
| 91|        91|21st Amendment|ZrSNKHiKK_PBB6UHx...|2013/08/16|   998|  2|  0|  0|2013-08-16|I'll be honest. I...|[ill, honest, exp...|[ill, honest, exp...|(8839,[4,13,18,32...|(8839,[4,13,18,32...|
+---+----------+--------------+--------------------+----------+------+---+---+---+----------+--------------------+--------------------+--------------------+--------------------+--------------------+
only 

In [ ]:
### TMP column_name= *

In [ ]:
# * LinearRegression

In [68]:
# regParam = Param(parent='undefined', name='regParam', doc='regularization parameter (>= 0).')¶
# maxIter = Param(parent='undefined', name='maxIter', doc='max number of iterations (>= 0).')
# elasticNetParam = Param(parent='undefined', name='elasticNetParam', doc='the ElasticNet mixing parameter, in range [0, 1]. For alpha = 0, the penalty is an L2 penalty. For alpha = 1, it is an L1 penalty.')
from pyspark.ml.regression import LinearRegression
lr1 = LinearRegression(featuresCol = 'tfidf_features', labelCol='level_1', maxIter=10, regParam=0.3, elasticNetParam=0.8)
lr1_model = lr1.fit(tfidf_result_training)

print("Coefficients: " + str(lr1_model.coefficients))
print("Intercept: " + str(lr1_model.intercept))

Coefficients: (8839,[614,796,1404,1534,1658,1893,3514,4218,4257,5076,7297,8131],[0.03512562533000398,0.005800134759622662,0.010991539330916312,0.003000184427165887,0.0005410077827142272,0.04100599499985956,0.013185241270330548,0.014420009600605958,0.029413751684438355,0.001827109819710285,0.01189240235441466,0.014740139576616039])
Intercept: 3.2669915194053667


In [69]:
trainingSummary_1 = lr1_model.summary
print("RMSE: %f" % trainingSummary_1.rootMeanSquaredError)
print("r2: %f" % trainingSummary_1.r2)

RMSE: 3.323097
r2: 0.018753


In [ ]:
#tfidf_result_training[['level_1']].describe().show()

In [70]:
lr1_predictions = lr1_model.transform(tfidf_result_test) 
lr1_predictions.select("prediction","level_1","tfidf_features").show(5)



+------------------+-------+--------------------+
|        prediction|level_1|      tfidf_features|
+------------------+-------+--------------------+
|3.2669915194053667|      2|(8839,[4,13,18,32...|
|3.2669915194053667|      1|(8839,[2,4,13,18,...|
|3.2669915194053667|      1|(8839,[2,4,13,18,...|
|3.6277011493913345|      1|(8839,[2,15,18,23...|
|3.4357864656610526|      1|(8839,[15,18,23,2...|
+------------------+-------+--------------------+
only showing top 5 rows



In [ ]:
# * RegressionEvaluator

In [71]:
from pyspark.ml.evaluation import RegressionEvaluator 
lr1_evaluator = RegressionEvaluator(predictionCol="prediction", labelCol="level_1",metricName="r2") 
print("R Squared (R2) on test data = %g" % lr1_evaluator.evaluate (lr1_predictions))

R Squared (R2) on test data = 0.0101553


In [72]:
test_result_1 = lr1_model.evaluate(tfidf_result_test) 
print("測試數據的均方根誤差 (RMSE) = %g" % test_result_1.rootMeanSquaredError)

測試數據的均方根誤差 (RMSE) = 3.48533


In [73]:
print("numIterations: %d" % trainingSummary_1.totalIterations) 
print("objectiveHistory: %s" % str(trainingSummary_1.objectiveHistory)) 
trainingSummary_1.residuals.show()

numIterations: 10
objectiveHistory: [0.4999716022036692, 0.49958641222170586, 0.4993562261763892, 0.49849455705163936, 0.49840087348899137, 0.49833571740712623, 0.4982947102164213, 0.49826553691414316, 0.49821586562308395, 0.49818991402678114, 0.49816429544176033]
+-------------------+
|          residuals|
+-------------------+
|-3.2669915194053667|
| 3.7330084805946333|
| 0.7330084805946333|
|-2.2669915194053667|
|-3.2669915194053667|
| 0.7330084805946333|
|-0.2669915194053667|
|-0.2669915194053667|
| 0.7330084805946333|
| 2.7330084805946333|
| 2.7330084805946333|
|-3.2669915194053667|
|  4.733008480594633|
| 0.7330084805946333|
|-2.2669915194053667|
|-2.2669915194053667|
|-2.2669915194053667|
| 0.7330084805946333|
|  4.733008480594633|
| 1.7330084805946333|
+-------------------+
only showing top 20 rows



In [74]:
predictions_1 = lr1_model.transform(tfidf_result_test)
predictions_1.select("prediction","level_1","tfidf_features").show()

+------------------+-------+--------------------+
|        prediction|level_1|      tfidf_features|
+------------------+-------+--------------------+
|3.2669915194053667|      2|(8839,[4,13,18,32...|
|3.2669915194053667|      1|(8839,[2,4,13,18,...|
|3.2669915194053667|      1|(8839,[2,4,13,18,...|
|3.6277011493913345|      1|(8839,[2,15,18,23...|
|3.4357864656610526|      1|(8839,[15,18,23,2...|
|3.2669915194053667|      0|(8839,[2,23,26,32...|
|3.2669915194053667|      1|(8839,[38,91,122,...|
|3.2669915194053667|      1|        (8839,[],[])|
|3.2669915194053667|      3|(8839,[18,22,26,3...|
|3.2669915194053667|     10|(8839,[4,7,8,9,10...|
|3.2669915194053667|     11|(8839,[4,7,8,9,10...|
|3.2669915194053667|      1|(8839,[4,7,8,9,10...|
|3.3965231313210253|      2|(8839,[8,9,27,39,...|
|3.3965231313210253|      4|(8839,[8,9,27,39,...|
|3.3398291237959117|      8|(8839,[0,7,8,13,1...|
|3.2669915194053667|      1|(8839,[46,70,75,1...|
|3.2669915194053667|      3|(8839,[46,70,75,1...|


In [ ]:
# * DecisionTreeRegressor

In [77]:
from pyspark.ml.regression import DecisionTreeRegressor
dt_1 = DecisionTreeRegressor(featuresCol ='tfidf_features', labelCol = 'level_1')
dt_model_1 = dt_1.fit(tfidf_result_training)
dt_predictions_1 = dt_model_1.transform(tfidf_result_training)

lr1_evaluator_1 = RegressionEvaluator(labelCol="level_1", predictionCol="prediction", metricName="rmse")
rmse_1 = lr1_evaluator_1.evaluate(dt_predictions_1)

In [76]:
print("Root Mean Squared Error (RMSE) on test data = %g" % rmse_1)

Root Mean Squared Error (RMSE) on test data = 3.22599


In [93]:
re_r2_1 = RegressionEvaluator(labelCol="level_1", predictionCol="prediction", metricName="r2")
re_rmse_1 = re_r2_1.evaluate(dt_predictions_1)
print(" r2 = %g" % re_rmse_1)

 r2 = 0.0752619


In [138]:
dt_model_1.featuresCol

Param(parent='DecisionTreeRegressor_647c986a08f8', name='featuresCol', doc='features column name.')

In [78]:
dt_model_1.featureImportances

SparseVector(8839, {2: 0.0333, 18: 0.0025, 82: 0.0827, 104: 0.0486, 189: 0.0141, 199: 0.0243, 256: 0.0022, 295: 0.0131, 614: 0.0521, 624: 0.0066, 648: 0.027, 761: 0.1122, 1069: 0.0601, 1404: 0.0368, 1534: 0.0198, 1550: 0.0033, 1651: 0.0178, 1903: 0.0062, 1918: 0.0097, 1935: 0.0201, 2015: 0.0979, 2094: 0.0058, 2203: 0.0036, 3514: 0.1683, 3551: 0.0211, 5491: 0.074, 5977: 0.0119, 7187: 0.0248})

In [133]:
dt_model_1.



TypeError: 'Param' object is not callable

In [124]:
dt_model_1.getLabelCol

<bound method HasLabelCol.getLabelCol of DecisionTreeRegressionModel: uid=DecisionTreeRegressor_647c986a08f8, depth=5, numNodes=57, numFeatures=8839>

In [ ]:
# * GBTRegressor

In [79]:
from pyspark.ml.regression import GBTRegressor
gbt_1 = GBTRegressor(featuresCol = 'tfidf_features', labelCol = 'level_1', maxIter=10)
gbt_model_1 = gbt_1.fit(tfidf_result_training)
gbt_predictions_1 = gbt_model_1.transform(tfidf_result_test)
gbt_predictions_1.select('prediction', 'level_1', 'tfidf_features').show(5)

+-----------------+-------+--------------------+
|       prediction|level_1|      tfidf_features|
+-----------------+-------+--------------------+
|3.794825074332152|      2|(8839,[4,13,18,32...|
|3.794825074332152|      1|(8839,[2,4,13,18,...|
|3.794825074332152|      1|(8839,[2,4,13,18,...|
|4.216373191332545|      1|(8839,[2,15,18,23...|
|4.216373191332545|      1|(8839,[15,18,23,2...|
+-----------------+-------+--------------------+
only showing top 5 rows



In [80]:
gbt_evaluator_1 = RegressionEvaluator(
    labelCol="level_1", predictionCol="prediction", metricName="rmse")
rmse_1 = gbt_evaluator_1.evaluate(gbt_predictions_1)
print("Root Mean Squared Error (RMSE) on test data = %g" % rmse_1)

Root Mean Squared Error (RMSE) on test data = 3.45207


In [96]:
gbt_evaluator_1_r2 = RegressionEvaluator(
    labelCol="level_1", predictionCol="prediction", metricName="r2")
gbt_r2_1 = gbt_evaluator_1_r2.evaluate(gbt_predictions_1)
print("Root Mean Squared Error (r2) on test data = %g" % gbt_r2_1)

Root Mean Squared Error (r2) on test data = 0.0289555


In [97]:
### TMP column_name= ** 

In [ ]:
# ** LinearRegression

In [99]:
# regParam = Param(parent='undefined', name='regParam', doc='regularization parameter (>= 0).')¶
# maxIter = Param(parent='undefined', name='maxIter', doc='max number of iterations (>= 0).')
# elasticNetParam = Param(parent='undefined', name='elasticNetParam', doc='the ElasticNet mixing parameter, in range [0, 1]. For alpha = 0, the penalty is an L2 penalty. For alpha = 1, it is an L1 penalty.')
from pyspark.ml.regression import LinearRegression
lr2 = LinearRegression(featuresCol = 'tfidf_features', labelCol='level_2', maxIter=10, regParam=0.3, elasticNetParam=0.8)
lr2_model = lr2.fit(tfidf_result_training)

print("Coefficients: " + str(lr2_model.coefficients))
print("Intercept: " + str(lr2_model.intercept))

Coefficients: (8839,[],[])
Intercept: 0.4085306980178339


In [100]:
trainingSummary_2 = lr2_model.summary
print("RMSE: %f" % trainingSummary_2.rootMeanSquaredError)
print("r2: %f" % trainingSummary_2.r2)

RMSE: 0.805306
r2: 0.000000


In [101]:
#tfidf_result_training[['level_1']].describe().show()

In [102]:
lr2_predictions = lr2_model.transform(tfidf_result_test) 
lr2_predictions.select("prediction","level_2","tfidf_features").show(5)


+------------------+-------+--------------------+
|        prediction|level_2|      tfidf_features|
+------------------+-------+--------------------+
|0.4085306980178339|      0|(8839,[4,13,18,32...|
|0.4085306980178339|      0|(8839,[2,4,13,18,...|
|0.4085306980178339|      0|(8839,[2,4,13,18,...|
|0.4085306980178339|      0|(8839,[2,15,18,23...|
|0.4085306980178339|      0|(8839,[15,18,23,2...|
+------------------+-------+--------------------+
only showing top 5 rows



In [ ]:
# ** RegressionEvaluator

In [103]:
from pyspark.ml.evaluation import RegressionEvaluator 
lr2_evaluator = RegressionEvaluator(predictionCol="prediction", labelCol="level_2",metricName="r2") 
print("R Squared (R2) on test data = %g" % lr2_evaluator.evaluate (lr2_predictions))

R Squared (R2) on test data = -0.000473625


In [104]:
test_result_2 = lr2_model.evaluate(tfidf_result_test) 
print("測試數據的均方根誤差 (RMSE) = %g" % test_result_2.rootMeanSquaredError)

測試數據的均方根誤差 (RMSE) = 0.831902


In [105]:
print("numIterations: %d" % trainingSummary_2.totalIterations) 
print("objectiveHistory: %s" % str(trainingSummary_2.objectiveHistory)) 
trainingSummary_2.residuals.show()

numIterations: 0
objectiveHistory: [0.4999716022036689]
+-------------------+
|          residuals|
+-------------------+
|-0.4085306980178339|
| 0.5914693019821661|
| 0.5914693019821661|
| 0.5914693019821661|
|-0.4085306980178339|
|-0.4085306980178339|
| 0.5914693019821661|
|-0.4085306980178339|
|-0.4085306980178339|
|-0.4085306980178339|
|-0.4085306980178339|
|-0.4085306980178339|
|-0.4085306980178339|
|-0.4085306980178339|
|-0.4085306980178339|
|  3.591469301982166|
|-0.4085306980178339|
| 0.5914693019821661|
|-0.4085306980178339|
| 0.5914693019821661|
+-------------------+
only showing top 20 rows



In [106]:
predictions_2 = lr2_model.transform(tfidf_result_test)
predictions_2.select("prediction","level_2","tfidf_features").show()

+------------------+-------+--------------------+
|        prediction|level_2|      tfidf_features|
+------------------+-------+--------------------+
|0.4085306980178339|      0|(8839,[4,13,18,32...|
|0.4085306980178339|      0|(8839,[2,4,13,18,...|
|0.4085306980178339|      0|(8839,[2,4,13,18,...|
|0.4085306980178339|      0|(8839,[2,15,18,23...|
|0.4085306980178339|      0|(8839,[15,18,23,2...|
|0.4085306980178339|      0|(8839,[2,23,26,32...|
|0.4085306980178339|      0|(8839,[38,91,122,...|
|0.4085306980178339|      0|        (8839,[],[])|
|0.4085306980178339|      1|(8839,[18,22,26,3...|
|0.4085306980178339|      1|(8839,[4,7,8,9,10...|
|0.4085306980178339|      1|(8839,[4,7,8,9,10...|
|0.4085306980178339|      0|(8839,[4,7,8,9,10...|
|0.4085306980178339|      0|(8839,[8,9,27,39,...|
|0.4085306980178339|      0|(8839,[8,9,27,39,...|
|0.4085306980178339|      0|(8839,[0,7,8,13,1...|
|0.4085306980178339|      0|(8839,[46,70,75,1...|
|0.4085306980178339|      0|(8839,[46,70,75,1...|


In [ ]:
# ** DecisionTreeRegressor

In [107]:
from pyspark.ml.regression import DecisionTreeRegressor
dt_2 = DecisionTreeRegressor(featuresCol ='tfidf_features', labelCol = 'level_2')
dt_model_2 = dt_2.fit(tfidf_result_training)
dt_predictions_2 = dt_model_2.transform(tfidf_result_training)

dt_evaluator_2 = RegressionEvaluator(labelCol="level_2", predictionCol="prediction", metricName="rmse")
rmse_2 = dt_evaluator_2.evaluate(dt_predictions_2)
print("Root Mean Squared Error (RMSE) on test data = %g" % rmse_2)

Root Mean Squared Error (RMSE) on test data = 0.760542


In [108]:
dt_evaluator_2 = RegressionEvaluator(labelCol="level_2", predictionCol="prediction", metricName="r2")
r2_2 = dt_evaluator_2.evaluate(dt_predictions_2)
print("r2 = %g" % r2_2)

r2 = 0.108082


In [109]:
dt_model_2.featureImportances

SparseVector(8839, {0: 0.0053, 2: 0.0066, 4: 0.0154, 10: 0.0089, 12: 0.0024, 55: 0.0012, 146: 0.0266, 175: 0.0322, 182: 0.0055, 352: 0.0219, 636: 0.0004, 640: 0.0718, 641: 0.0226, 657: 0.0496, 770: 0.0099, 870: 0.0656, 3128: 0.0756, 3264: 0.0902, 3695: 0.0283, 4027: 0.2988, 5184: 0.0399, 8131: 0.0323, 8191: 0.0335, 8491: 0.0555})

In [ ]:
# ** GBTRegressor

In [110]:
from pyspark.ml.regression import GBTRegressor
gbt_2 = GBTRegressor(featuresCol = 'tfidf_features', labelCol = 'level_2', maxIter=10)
gbt_model_2 = gbt_2.fit(tfidf_result_training)
gbt_predictions_2 = gbt_model_2.transform(tfidf_result_test)
gbt_predictions_2.select('prediction', 'level_2', 'tfidf_features').show(5)

+-------------------+-------+--------------------+
|         prediction|level_2|      tfidf_features|
+-------------------+-------+--------------------+
|0.34219589002766915|      0|(8839,[4,13,18,32...|
|0.34219589002766915|      0|(8839,[2,4,13,18,...|
|0.34219589002766915|      0|(8839,[2,4,13,18,...|
|0.38877182480774225|      0|(8839,[2,15,18,23...|
|0.38877182480774225|      0|(8839,[15,18,23,2...|
+-------------------+-------+--------------------+
only showing top 5 rows



In [111]:
gbt_evaluator_2 = RegressionEvaluator(
    labelCol="level_2", predictionCol="prediction", metricName="rmse")
rmse_2 = gbt_evaluator_2.evaluate(gbt_predictions_2)
print("Root Mean Squared Error (RMSE) on test data = %g" % rmse_2)

Root Mean Squared Error (RMSE) on test data = 0.797634


In [112]:
gbt_evaluator_2 = RegressionEvaluator(
    labelCol="level_2", predictionCol="prediction", metricName="r2")
r2_2_2 = gbt_evaluator_2.evaluate(gbt_predictions_2)
print("r2 = %g" % r2_2_2)

r2 = 0.0802534


In [ ]:
### TMP column_name= ***

In [ ]:
# *** LinearRegression

In [81]:
# regParam = Param(parent='undefined', name='regParam', doc='regularization parameter (>= 0).')¶
# maxIter = Param(parent='undefined', name='maxIter', doc='max number of iterations (>= 0).')
# elasticNetParam = Param(parent='undefined', name='elasticNetParam', doc='the ElasticNet mixing parameter, in range [0, 1]. For alpha = 0, the penalty is an L2 penalty. For alpha = 1, it is an L1 penalty.')
from pyspark.ml.regression import LinearRegression
lr3 = LinearRegression(featuresCol = 'tfidf_features', labelCol='level_3', maxIter=10, regParam=0.3, elasticNetParam=0.8)
lr3_model = lr3.fit(tfidf_result_training)

print("Coefficients: " + str(lr3_model.coefficients))
print("Intercept: " + str(lr3_model.intercept))

Coefficients: (8839,[],[])
Intercept: 0.7199409325836316


In [82]:
trainingSummary_3 = lr3_model.summary
print("RMSE: %f" % trainingSummary_3.rootMeanSquaredError)
print("r2: %f" % trainingSummary_3.r2)

RMSE: 1.174404
r2: 0.000000


In [ ]:
#tfidf_result_training[['level_1']].describe().show()

In [83]:
lr3_predictions = lr3_model.transform(tfidf_result_test) 
lr3_predictions.select("prediction","level_3","tfidf_features").show(5)


+------------------+-------+--------------------+
|        prediction|level_3|      tfidf_features|
+------------------+-------+--------------------+
|0.7199409325836316|      0|(8839,[4,13,18,32...|
|0.7199409325836316|      0|(8839,[2,4,13,18,...|
|0.7199409325836316|      0|(8839,[2,4,13,18,...|
|0.7199409325836316|      0|(8839,[2,15,18,23...|
|0.7199409325836316|      1|(8839,[15,18,23,2...|
+------------------+-------+--------------------+
only showing top 5 rows



In [ ]:
# *** RegressionEvaluator

In [84]:
from pyspark.ml.evaluation import RegressionEvaluator 
lr3_evaluator = RegressionEvaluator(predictionCol="prediction", labelCol="level_3",metricName="r2") 
print("R Squared (R2) on test data = %g" % lr3_evaluator.evaluate (lr3_predictions))

R Squared (R2) on test data = -0.000393047


In [85]:
test_result_3 = lr3_model.evaluate(tfidf_result_test) 
print("測試數據的均方根誤差 (RMSE) = %g" % test_result_3.rootMeanSquaredError)

測試數據的均方根誤差 (RMSE) = 1.24407


In [86]:
print("numIterations: %d" % trainingSummary_3.totalIterations) 
print("objectiveHistory: %s" % str(trainingSummary_3.objectiveHistory)) 
trainingSummary_3.residuals.show()

numIterations: 0
objectiveHistory: [0.4999716022036691]
+-------------------+
|          residuals|
+-------------------+
|-0.7199409325836316|
|0.28005906741636843|
| 1.2800590674163685|
|0.28005906741636843|
|0.28005906741636843|
|0.28005906741636843|
|0.28005906741636843|
| 1.2800590674163685|
| 2.2800590674163685|
|0.28005906741636843|
|0.28005906741636843|
|-0.7199409325836316|
|-0.7199409325836316|
|-0.7199409325836316|
|0.28005906741636843|
|-0.7199409325836316|
|0.28005906741636843|
|0.28005906741636843|
| 1.2800590674163685|
|0.28005906741636843|
+-------------------+
only showing top 20 rows



In [87]:
predictions_3 = lr3_model.transform(tfidf_result_test)
predictions_3.select("prediction","level_3","tfidf_features").show()

+------------------+-------+--------------------+
|        prediction|level_3|      tfidf_features|
+------------------+-------+--------------------+
|0.7199409325836316|      0|(8839,[4,13,18,32...|
|0.7199409325836316|      0|(8839,[2,4,13,18,...|
|0.7199409325836316|      0|(8839,[2,4,13,18,...|
|0.7199409325836316|      0|(8839,[2,15,18,23...|
|0.7199409325836316|      1|(8839,[15,18,23,2...|
|0.7199409325836316|      0|(8839,[2,23,26,32...|
|0.7199409325836316|      0|(8839,[38,91,122,...|
|0.7199409325836316|      1|        (8839,[],[])|
|0.7199409325836316|      3|(8839,[18,22,26,3...|
|0.7199409325836316|      2|(8839,[4,7,8,9,10...|
|0.7199409325836316|      0|(8839,[4,7,8,9,10...|
|0.7199409325836316|      0|(8839,[4,7,8,9,10...|
|0.7199409325836316|      0|(8839,[8,9,27,39,...|
|0.7199409325836316|      1|(8839,[8,9,27,39,...|
|0.7199409325836316|      1|(8839,[0,7,8,13,1...|
|0.7199409325836316|      1|(8839,[46,70,75,1...|
|0.7199409325836316|      0|(8839,[46,70,75,1...|


In [ ]:
# *** DecisionTreeRegressor

In [88]:
from pyspark.ml.regression import DecisionTreeRegressor
dt_3 = DecisionTreeRegressor(featuresCol ='tfidf_features', labelCol = 'level_3')
dt_model_3 = dt_3.fit(tfidf_result_training)
dt_predictions_3 = dt_model_3.transform(tfidf_result_training)

dt_evaluator_3 = RegressionEvaluator(labelCol="level_3", predictionCol="prediction", metricName="rmse")
rmse_3 = dt_evaluator_3.evaluate(dt_predictions_3)

In [89]:
print("Root Mean Squared Error (RMSE) on test data = %g" % rmse_3)

Root Mean Squared Error (RMSE) on test data = 1.14217


In [88]:
# to do
dt_evaluator_3_r2 = RegressionEvaluator(labelCol="level_3", predictionCol="prediction", metricName="r2")
rmse_3_r2 = dt_evaluator_3_r2.evaluate(dt_predictions_3)
print("r2 = %g" % rmse_3_r2)

In [90]:
dt_model_3.featureImportances

SparseVector(8839, {2: 0.0103, 52: 0.0449, 69: 0.0004, 190: 0.005, 252: 0.151, 441: 0.0094, 491: 0.0632, 864: 0.1097, 960: 0.0914, 1065: 0.011, 1159: 0.0262, 1188: 0.0122, 1233: 0.0517, 1271: 0.0525, 1432: 0.0243, 1641: 0.0253, 1708: 0.0348, 2378: 0.0913, 4027: 0.0407, 4194: 0.0058, 4228: 0.0093, 4610: 0.0452, 4934: 0.0352, 4998: 0.034, 6269: 0.0019, 8479: 0.0133})

In [ ]:
# *** GBTRegressor

In [91]:
from pyspark.ml.regression import GBTRegressor
gbt_3 = GBTRegressor(featuresCol = 'tfidf_features', labelCol = 'level_3', maxIter=10)
gbt_model_3 = gbt_3.fit(tfidf_result_training)
gbt_predictions_3 = gbt_model_3.transform(tfidf_result_test)
gbt_predictions_3.select('prediction', 'level_3', 'tfidf_features').show(5)

+-------------------+-------+--------------------+
|         prediction|level_3|      tfidf_features|
+-------------------+-------+--------------------+
| 0.7369319784626313|      0|(8839,[4,13,18,32...|
| 0.8950276646704375|      0|(8839,[2,4,13,18,...|
| 0.8950276646704375|      0|(8839,[2,4,13,18,...|
|0.47002940038540025|      0|(8839,[2,15,18,23...|
|0.37664135928274906|      1|(8839,[15,18,23,2...|
+-------------------+-------+--------------------+
only showing top 5 rows



In [92]:
gbt_evaluator_3 = RegressionEvaluator(
    labelCol="level_3", predictionCol="prediction", metricName="rmse")
rmse_3 = gbt_evaluator_3.evaluate(gbt_predictions_3)
print("Root Mean Squared Error (RMSE) on test data = %g" % rmse_3)

Root Mean Squared Error (RMSE) on test data = 1.22615


In [98]:
gbt_evaluator_3_r2 = RegressionEvaluator(
    labelCol="level_3", predictionCol="prediction", metricName="r2")
rmse_3_r2 = gbt_evaluator_3_r2.evaluate(gbt_predictions_3)
print("r2 = %g" % rmse_3_r2)

r2 = 0.028227


In [ ]:
# RDD years predit tmp***count
# find DecisionTreeRegressor category to words !!!
# GPU setting step
# R2